# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
#загрузка библиотек
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats as st

## Загрузка и подготовка данных

In [2]:
try:
    d0 = pd.read_csv('datasets/geo_data_0.csv')
    d1 = pd.read_csv('datasets/geo_data_1.csv')
    d2 = pd.read_csv('datasets/geo_data_2.csv')
except:
    d0 = pd.read_csv('/datasets/geo_data_0.csv')
    d1 = pd.read_csv('/datasets/geo_data_1.csv')
    d2 = pd.read_csv('/datasets/geo_data_2.csv')
    
data = [d0, d1, d2]

In [3]:
#Проведем краткое описание датасета
def describe(d):
    print('Имя датасета', [x for x in globals() if globals()[x] is d][0])
    print('Размерность', d.shape)
    print('Количество NA', d.isna().sum().sum())
    print('Всего дубликатов', d.duplicated().sum())
    print('Не уникальных названий скважин', d['id'].duplicated().sum())
    print('Описание датасета')
    print(d.describe())

In [4]:
for s in data:
    describe(s)
    print('--------------')


Имя датасета d0
Размерность (100000, 5)
Количество NA 0
Всего дубликатов 0
Не уникальных названий скважин 10
Описание датасета
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
--------------
Имя датасета d1
Размерность (100000, 5)
Количество NA 0
Всего дубликатов 0
Не уникальных названий скважин 4
Описание датасета
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean  

Выводы по краткому анализу:
1. Все данные заполнены
2. Есть очень небольшие повторения по названием скважин. Я буду считать, что это ошибка ввода и не буду исправлять
3. Все данные требуют нормирования: имеется разброс между минимальными и максимальными значениями показателей

## Обучение и проверка модели

In [5]:
def train_val_model(d, report):
    
    features, target = d.drop(['product', 'id'], axis=1), d["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=12345)
    #нормализуекм данные, не смешивая выборки
    scaler = MinMaxScaler()
    
    #Закомментил и исправил ниже в соответствии с замечаниями
    #features_train = scaler.fit_transform(features_train)
    #features_valid = scaler.fit_transform(features_valid)
    
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**.5
    average_product = sum(predictions) / len(predictions)
    
    if report == True:
        print("RMSE: {0:.2f}".format(rmse))
        print("Average product: {0:.2f}".format(average_product))
    
    return (predictions, target_valid.reset_index(drop=True))
    

In [6]:
for s in data:
    print('Имя датасета', [x for x in globals() if globals()[x] is s][0])
    train_val_model(s, True)
    print('--------------')

Имя датасета d0
RMSE: 37.58
Average product: 92.59
--------------
Имя датасета d1
RMSE: 0.89
Average product: 68.73
--------------
Имя датасета d2
RMSE: 40.03
Average product: 94.97
--------------


Вывод: лучше всего мы спрогнозировали датасет d1, т.к. у него наименьший RMSE

## Подготовка к расчёту прибыли

In [14]:
#Сохраним в переменные
predictions = []
targets = []
for s in data:
    predictions.append(train_val_model(s, False)[0])
    targets.append(train_val_model(s, False)[1])


#Условия задачи
BUDGET = 10000000000
BARREL_COST = 450*1000

print('Точка безубыточности : {0:.2f}'.format(BUDGET/BARREL_COST/200))

#Вывод: необходимо добывать не менее 111.11 баррелей, чтобы "выйти в ноль"

Точка безубыточности : 111.11


In [9]:
# Сравним синтетически посчитанную точку безубыточности со средним значением в каждом регионе
for s in data:
    print('Имя датасета', [x for x in globals() if globals()[x] is s][0])
    print('Среднее значение в регионе : {0:.2f}'.format(s['product'].mean()), 'баррелей')
    print('Точка безубыточности : {0:.2f}'.format(BUDGET/BARREL_COST/200), 'баррелей')
    print('--------------')

Имя датасета d0
Среднее значение в регионе : 92.50 баррелей
Точка безубыточности : 111.11 баррелей
--------------
Имя датасета d1
Среднее значение в регионе : 68.83 баррелей
Точка безубыточности : 111.11 баррелей
--------------
Имя датасета d2
Среднее значение в регионе : 95.00 баррелей
Точка безубыточности : 111.11 баррелей
--------------


Вывод: По всем регионам среднее значение по скважине меньше точки безубыточности.

## Расчёт прибыли и рисков 

In [11]:
def profit(predictions, target):
    top_preds = predictions.sort_values(ascending=False)
    top_target = target[top_preds.index][:200]
    profit = top_target.sum() * BARREL_COST
    return profit - BUDGET


In [12]:
def interval_risk(predictions, target):
    state = np.random.RandomState(12345)
    revenue = []
    for _ in range(1000):
        target_sample = target.sample(500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(profit(predictions_sample, target_sample))   
    
    revenue = pd.Series(revenue)
    low = revenue.quantile(0.025)
    high= revenue.quantile(0.975)
    
    mean_revenue = int(sum(revenue) / len(revenue))
    risk = st.percentileofscore(revenue, 0)

    return ((low, high), mean_revenue, risk)


In [13]:

for i in range(3):
    interval, mean_revenue, risk = interval_risk(predictions[i], targets[i])
    print(f"data {i}:")
    print(f"Mean: {mean_revenue}")
    print(f"95% confidence interval: {interval}")
    print(f"Risk of loss: {risk}")
    print("-----------------------------------------\n\n")
    

data 0:
Mean: 425938526
95% confidence interval: (-102090094.83793654, 947976353.3583689)
Risk of loss: 6.0
-----------------------------------------


data 1:
Mean: 515222773
95% confidence interval: (68873225.37050176, 931547591.2570494)
Risk of loss: 1.0
-----------------------------------------


data 2:
Mean: 435008362
95% confidence interval: (-128880547.32978901, 969706954.1802661)
Risk of loss: 6.4
-----------------------------------------




Вывод:
1. Больше всего прогнозируемая средняя прибыль по датасету 1, с наименьшим риском. Этот регион и необходимо выбрать
2. По датасету 1 с 95% вероятностью можно сказать, что прибыль будет находиться в интервале 68873225.37 до 931547591.25
3. По датасетам 0 и 2 доверительный интервал от глубоких убытков до большой прибыли не дает возможности полноценно спронозировать работу. Поэтому эти регионы либо следует уточнить, либо исключить из дальнейшего рассмотрения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [X]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Выполнен шаг 1: данные подготовлены
- [X]  Выполнен шаг 2: модели обучены и проверены
    - [X]  Данные корректно разбиты на обучающую и валидационную выборки
    - [X]  Модели обучены, предсказания сделаны
    - [X]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [X]  На экране напечатаны результаты
    - [X]  Сделаны выводы
- [X]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [X]  Для всех ключевых значений созданы константы Python
    - [X]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [X]  По предыдущему пункту сделаны выводы
    - [X]  Написана функция расчёта прибыли
- [X]  Выполнен шаг 4: посчитаны риски и прибыль
    - [X]  Проведена процедура *Bootstrap*
    - [X]  Все параметры бутстрепа соответствуют условию
    - [X]  Найдены все нужные величины
    - [X]  Предложен регион для разработки месторождения
    - [X]  Выбор региона обоснован